# The selfish example

Let's have a look at a very simple example with a blocking implementation that can be broken up to allow concurrency.

Consider `time.sleep` in that context to be just a simple placeholder for other more complex work with a blocking nature.

In [1]:
import asyncio
import time

In [2]:
# override print to allow showing the output in one cell as it was printed in a terminal
import os

real_print = print
lines = []
def print(text):
    global lines
    lines.append(text)

Here's the blocking code to work on. Just a `time.sleep` statement to wait for `wait_for` seconds.

In [3]:
async def blocking_wait_for(wait_for):
    print(f"I'm going to sleep right now for {wait_for} seconds.")
    
    tic = time.perf_counter()
    time.sleep(wait_for)
    toc = time.perf_counter()
    
    print(
        "Hey y'all. That was a good nap! "
        f"Slept for {toc-tic:.2f} seconds. "
        "Did i miss something?"
    )

And here's a simple notifier. We would love to see some message from it every now and then.

In [4]:
async def notify_every(every):
    while keep_running:
        print("Notify!")
        await asyncio.sleep(every)

The `exit_after` function allows us to end execution of all the work after a chosen time.

In [5]:
async def exit_after(exit_after):
    global keep_running
    
    await asyncio.sleep(exit_after)
    keep_running = False

And heres the final bit. We wait for the resut of every function using the `asyncio.gather` functiom. Let's see how it behaves

In [6]:
keep_running = True
exec_for = 5
wait_for = 2
_notify_every = .2

tic = time.perf_counter()
await asyncio.gather(
    exit_after(exec_for),
    notify_every(_notify_every),
    blocking_wait_for(wait_for),
)
toc = time.perf_counter()

print(f"All over execution time was {toc - tic:.2f} seconds.")

In [7]:
real_print("\n".join(lines))
lines = []

Notify!
I'm going to sleep right now for 2 seconds.
Hey y'all. That was a good nap! Slept for 2.00 seconds. Did i miss something?
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
All over execution time was 5.02 seconds.


That was a forseable outcome. `time.sleep` blocked the exeution globally, hence everything else was executed only after the `blocking_wait_for` finished.

Let's check if that already helped.